### API for coordinates

In [3]:
import requests
import numpy as np
import pandas as pd
import json

In [14]:
address = input("Enter the  address: ") 

Enter the  address: Sint-Pietersvliet 7 2000 Antwerpen


In [15]:
def get_coordinates(address: str):
    req = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    info = {'address' : address, 
                'x_value' : req['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : req['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : req['LocationResult'][0]['Thoroughfarename'],
                'house_number' : req['LocationResult'][0]['Housenumber'], 
                'postcode': req['LocationResult'][0]['Zipcode'], 
                'municipality' : req['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
    return info['polygon'][0]['coordinates'][0] 

In [16]:
adresss_coordination = get_coordinates(address)
adresss_coordination

[[152234.7539967969, 212884.87605375424],
 [152234.82894080132, 212884.9189977534],
 [152234.7067647949, 212885.13282175362],
 [152234.54202879965, 212885.42107775435],
 [152221.9368447885, 212878.215253748],
 [152216.66906878352, 212875.20392574742],
 [152215.47796478122, 212874.52302974463],
 [152215.7520127818, 212874.04302974418],
 [152215.85204478353, 212874.10197374597],
 [152216.04097278416, 212873.7709657438],
 [152225.4219647944, 212857.33397373557],
 [152225.34394878894, 212857.28994173557],
 [152231.24206079543, 212846.96808572486],
 [152231.36404479295, 212847.0380377248],
 [152246.52404480428, 212820.42696570978],
 [152246.43796480447, 212820.37493370846],
 [152246.70881280303, 212819.90990970656],
 [152246.73498880863, 212819.8649817072],
 [152246.9969408065, 212820.01717370749],
 [152247.5319168046, 212820.3279577084],
 [152249.9582208097, 212816.06210170686],
 [152251.11796481162, 212814.0229977034],
 [152251.02298881114, 212814.00994170457],
 [152251.31399680674, 21281